In [3]:
import pandas as pd

In [4]:
uni_df = pd.read_excel('./csp.xlsm', sheet_name='Uni', usecols='A:D', dtype='str')

In [5]:
course_df = pd.read_excel('./csp.xlsm', sheet_name='Course', dtype='str')
course_df['Updated'] = course_df['Updated'].str.split(' ').str[0]

course_df = course_df.drop('FoE Name', axis=1) # drop the FoE name which we will reconstruct later

In [6]:
foe_dict = {}
foe_dict['asced2'] = pd.read_excel('./csp.xlsm', sheet_name='FoE', usecols='A:B', dtype='str').dropna()
foe_dict['asced4'] = pd.read_excel('./csp.xlsm', sheet_name='FoE', usecols='D:E', dtype='str').dropna()
foe_dict['asced6'] = pd.read_excel('./csp.xlsm', sheet_name='FoE', usecols='G:H', dtype='str').dropna()

In [7]:
def title_except_and(text):
    words = text.split()
    return ' '.join([w.capitalize() if w.lower() != 'and' else 'and' for w in words])

foe_dict['asced2']['Name'] = foe_dict['asced2']['Name'].apply(title_except_and)

In [8]:
foe_dict['asced6']['ASCED 4'] = foe_dict['asced6']['ASCED 6'].str[:4]
foe_dict['asced6']['ASCED 2'] = foe_dict['asced6']['ASCED 6'].str[:2]


foe_df = (foe_dict['asced6']
          .merge(foe_dict['asced4'], how='left', on='ASCED 4')
          .merge(foe_dict['asced2'], how='left', on='ASCED 2')
)

foe_df = (foe_df
          .rename(columns={'ASCED 6':'asced', 'Name.2':'FoE_asced6', 'Name.1':'FoE_asced4', 'Name':'FoE_asced2'})
          .reindex(columns=['asced','FoE_asced6', 'FoE_asced4', 'FoE_asced2'])
)

In [9]:
output_df = (
    course_df
    .merge(foe_df, how='left', left_on='FoE', right_on='asced')
    .merge(uni_df, how='left', left_on='Uni ID', right_on='ID')
    .reindex(columns=['Course', 'Description', 'Duration', 'Location', 'URL_x', 'Updated', 'FoE', 'FoE_asced6', 'FoE_asced4', 'FoE_asced2', 'State', 'Name'])
    .rename(columns={'Name':'Uni', 'URL_x':'URL'})
)

output_df = output_df.fillna('')

In [10]:
output_df

,Course,Description,Duration,Location,URL,Updated,FoE,FoE_asced6,FoE_asced4,FoE_asced2,State,Uni
0,Master of Teaching (Primary),The Master of Teaching (Primary) (MTch (Prim))...,Full time: 2 years,Lake Macquarie,https://www.avondale.edu.au/course/master-of-t...,2025-03-27,070103,Teacher Education: Primary,Teacher Education,Education,NSW,Avondale University
1,Master of Teaching (Secondary),The Master of Teaching (Secondary) (MTch (Sec)...,Full time: 2 years,Lake Macquarie,https://www.avondale.edu.au/course/master-of-t...,2025-03-27,070105,Teacher Education: Secondary,Teacher Education,Education,NSW,Avondale University
2,Graduate Certificate in Agricultural Business ...,Gain a solid foundation for your agribusiness ...,Minimum time - 0.5 year(s),Online,https://study.csu.edu.au/courses/graduate-cert...,2025-03-27,080321,Farm Management and Agribusiness,Business and Management,Management and Commerce,NSW,Charles Sturt University
3,Graduate Certificate in Agriculture,The Graduate Certificate in Agriculture from C...,Minimum time - 0.5 year(s),Online,https://study.csu.edu.au/courses/graduate-cert...,2025-03-27,050101,Agricultural Science,Agriculture,"Agriculture, Environmental and Related Studies",NSW,Charles Sturt University
4,Graduate Certificate in Environmental Management,Do you want to be a changemaker for a sustaina...,Minimum time - 0.5 year(s),Online,https://study.csu.edu.au/courses/graduate-cert...,2025-03-27,050901,"Land, Parks and Wildlife Management",Environmental Studies,"Agriculture, Environmental and Related Studies",NSW,Charles Sturt University
...,...,...,...,...,...,...,...,...,...,...,...,...
1897,Master of Teaching (Secondary)/Graduate Certif...,"This is a graduate entry, preservice teacher e...",2 years full-time or equivalent part-time,"Brisbane, Melbourne, Online, Strathfield",https://www.acu.edu.au/course/master-of-teachi...,2025-03-31,070105,Teacher Education: Secondary,Teacher Education,Education,National,Australian Catholic University
1898,Graduate Certificate in Supervision,Supervision is the discipline of enabling prof...,1 year part-time,Online,https://www.acu.edu.au/course/graduate-certifi...,2025-03-31,070105,Teacher Education: Secondary,Teacher Education,Education,National,Australian Catholic University
1899,Graduate Diploma in Spiritual Direction,Spiritual Direction is the process and practic...,2 years part-time,Online,https://www.acu.edu.au/course/graduate-diploma...,2025-03-31,091703,Religious Studies,Philosophy and Religious Studies,Society and Culture,National,Australian Catholic University
1900,Graduate Diploma in Supervision,–,–,Online,https://www.acu.edu.au/course/graduate-diploma...,2025-03-31,080399,"Business and Management, n.e.c.",Business and Management,Management and Commerce,National,Australian Catholic University


In [11]:
# Convert the DataFrame to the required JSON format
json_output = {
    "data": output_df.to_dict(orient="records")  # Convert each row to a dictionary
}

# Print or save the JSON output
import json
with open("csp.json", "w") as f:
    json.dump(json_output, f, indent=4)